## Import Twitter data using Tweepy

In [ ]:
## Import Tweepy to analyze Twitter data with Python
import tweepy
from string import punctuation
## Import the Natural Language Toolkit
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

## Connect to Twitter Data using the Twitter API

In [ ]:
# Connect Jupyter to your Crowdmark API key.
# Please place your Crowdmark API key somewhere and link to it by adjusting the route below.
# The API key allows the computer hosting your Jupyter notebook to programmatically access data from Crowdmark.
with open("/home/jcollian/.twitter-keys", 'r') as f:
    Twitter_Keys = f.read()
# apiKey

In [ ]:
type(Twitter_Keys)

Hack: Define a dictionary with `keys = {...}` using the content from Twitter_Keys. The dictionary will populate your Twitter keys below.

In [ ]:
# == OAuth Authentication ==
#
# This mode of authentication is the new preferred way
# of authenticating with Twitter.
# Source: https://github.com/tweepy/tweepy/blob/master/examples/oauth.py

# The consumer keys can be found on your application's Details
# page located at https://dev.twitter.com/apps (under "OAuth settings")

consumer_key=keys[consumer_key]
consumer_secret= keys[consumer_secret]

# The access tokens can be found on your applications's Details
# page located at https://dev.twitter.com/apps (located
# under "Your access token")
access_token=keys[access_token]
access_token_secret=keys[access_token_secret]

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.secure = True
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

# If the authentication was successful, you should
# see the name of the account print out
print(api.me().name)

## Select the Twitter Account to Analyze

In [ ]:
screenName = 'pimsmath'
tweetsPerRetrieval = 50

# return list of Status object instances
data = api.user_timeline(
    screen_name=screenName,
    count=tweetsPerRetrieval,
    tweet_mode='extended', # avoid truncation
    include_rts=True # include retweets
)

In [ ]:
# Collect recent tweets
tweets = [] + data # extend first tweets
lastIndex = tweets[-1].id - 1 # get last tweet index

# recover tweets
while len(data) > 0:
    data = api.user_timeline(
        screen_name=screenName,
        count=tweetsPerRetrieval,
        max_id=lastIndex, # prevent duplicates
        tweet_mode='extended',
        include_rts=True,
    )
    tweets += data
    lastIndex = tweets[-1].id - 1
print('Retrieved %d tweets!' % len(tweets))

In [ ]:
# isolate mentions, get names
mentions = [m['screen_name']
                for t in tweets
                    for m in t.entities['user_mentions']]
mentions[:5] # slice first five

In [ ]:
# isolate media, get each URL
remedia = [m['media_url']
               for t in tweets
                   if hasattr(t, 'retweeted_status')
                   and 'media' in t.retweeted_status.entities
                       for m in t.retweeted_status.entities['media']]

media = remedia + [m['media_url']
                       for t in tweets
                           if 'media' in t.entities 
                               for m in t.entities['media']]
media[:10]

In [ ]:
# prefer retweet data, get tweet text
text = [t.retweeted_status.full_text
            if hasattr(t, 'retweeted_status')
            else t.full_text
                for t in tweets]
text[:10]

In [ ]:
# tokenize words
tokens = []
for t in text:
    for w in t.replace('\n', '').lower().split(' '): # remove newlines, split on spaces
        try:
            if (
                not (w.startswith('http') or w.startswith('@')) # ignore if URL or mention
                and w not in stopwords.words('english') # ignore stopwords
                and str(w).translate(punctuation) != '' # ignore punctuation
                and w != u''): # ignore empties
                    tokens.append(str(w).translate(punctuation))
        except UnicodeEncodeError: pass # ignore if not ascii
tokens[:10]

In [ ]:
# get word frequency
freq = dict((t, tokens.count(t)) for t in tokens)
sorted(freq, key=freq.get)[-20:] # slice frequent words